In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph, Namespace, Literal, RDF, OWL

def fetch_civilization_leaders():
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    query = """
    SELECT ?leader ?leaderLabel ?civilization ?civilizationLabel ?description
    WHERE {
      ?leader wdt:P31 wd:Q5;
              wdt:P1441 wd:Q24050047;
              rdfs:label ?leaderLabel.
      FILTER(LANG(?leaderLabel) = "en")

      OPTIONAL {
        ?leader wdt:P27 ?civilization.
        ?civilization rdfs:label ?civilizationLabel.
        FILTER(LANG(?civilizationLabel) = "en")
      }
      
      OPTIONAL { 
        ?leader schema:description ?description.
        FILTER(LANG(?description) = "en")
      }
    }
    LIMIT 100
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"]

In [2]:
wikidata_leaders = fetch_civilization_leaders()

In [3]:
len(wikidata_leaders)

27

In [4]:
wikidata_leaders

[{'leader': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q79789'},
  'leaderLabel': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'Frédéric Barberousse'},
  'civilization': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q12548'},
  'civilizationLabel': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'Holy Roman Empire'},
  'description': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'Holy Roman Emperor from 1155 to 1190'}},
 {'leader': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q80398'},
  'leaderLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Pericles'},
  'civilization': {'type': 'uri',
   'value': 'http://www.wikidata.org/entity/Q844930'},
  'civilizationLabel': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'Classical Athens'},
  'description': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'Athenian statesman, orator and general (c. 495 – 429 BC)'}},
 {'leader': {'type': 'uri', 'value': 'http://www.wikid

In [5]:
def process_leaders_into_ontology(wikidata_leaders, ns: Namespace, wikidata_ns: Namespace, g: Graph):
    for leader in wikidata_leaders:
        leader_name = leader["leaderLabel"]["value"].replace(" ", "_")
        leader_individual = ns[leader_name]

        g.add((leader_individual, RDF.type, ns.Leader))
        g.add((leader_individual, ns.name, Literal(leader["leaderLabel"]["value"])))

        g.add((leader_individual, ns.source, Literal("wikidata")))

        if "leader" in leader:
            g.add((leader_individual, OWL.sameAs, wikidata_ns[leader["leader"]["value"].split("/")[-1]]))

        if "description" in leader:
            g.add((leader_individual, ns.description, Literal(leader["description"]["value"])))

        if "civilization" in leader and "civilizationLabel" in leader:
            civ_name = leader["civilizationLabel"]["value"].replace(" ", "_")
            civ_individual = ns[civ_name]

            if (civ_individual, None, None) not in g:
                g.add((civ_individual, RDF.type, ns.Civilization))
                g.add((civ_individual, ns.name, Literal(leader["civilizationLabel"]["value"])))

            g.add((leader_individual, ns.rules, civ_individual))

            g.add((civ_individual, OWL.sameAs, wikidata_ns[leader["civilization"]["value"].split("/")[-1]]))
        else:
            g.add((leader_individual, ns.rules, Literal("None")))

In [6]:
ns = Namespace("http://n4d.sh/civ6#")
wikidata_ns = Namespace("http://www.wikidata.org/entity/")
g = Graph()
g.parse("civ6-ontology-v2.rdf", format="xml")

wikidata_leaders = fetch_civilization_leaders()

In [7]:
process_leaders_into_ontology(wikidata_leaders, ns, wikidata_ns, g)

g.serialize(destination="civ6-ontology-v2.rdf", format="xml")
print("Ontology updated with leaders from Wikidata.")

Ontology updated with leaders from Wikidata.
